In [39]:
import nltk

from sklearn.utils import shuffle

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, TimeDistributed, Bidirectional
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout
from keras.utils import to_categorical

import numpy as np
from numpy import array

Using TensorFlow backend.


In [21]:
import re

In [43]:
corp = open('/media/sf_communalflat/ihatelinguistics/history.conll', encoding='utf-8').read()

from conllu import parse, parse_tree
from conllu import print_tree

tree = parse(corp)

import re
p = re.compile('[а-яА-ЯёЁ]')
p1=re.compile('[0-9]')

lemmas=[]
for i in tree:
    for j in i:
        if p.search(j['lemma']) and not p1.search(j['lemma']):
            lemmas.append(j['lemma'].lower())

In [149]:
from numpy import array

def generate_seq(model, tokenizer, max_length, seed_text, n_words):
    in_text = seed_text
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        yhat = model.predict_classes(encoded, verbose=0)
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        in_text += ' ' + out_word
    return in_text

In [64]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([' '.join(i for i in lemmas)])
encoded = tokenizer.texts_to_sequences([' '.join(i for i in lemmas)])[0]
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
sequences = list()
for i in range(2, len(encoded)):
    sequence = encoded[i-2:i+1]
    sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=50, verbose=2)

Vocabulary Size: 8134
Total Sequences: 64202
Max Sequence Length: 3


MemoryError: 

In [46]:
#model.save('lemmas_model.h5')

In [68]:
from keras.models import load_model
model2=load_model('lemmas_model.h5')

In [150]:
#так генерируется наиболее вероятное следующеее слово
print(generate_seq(model2, tokenizer, max_length-1, 'взгляд православный', 1))

взгляд православный верующий


In [99]:
#получаем матрицу вероятностей появления для всех известных модели слов после заданного н-грамма
def get_all_probs(model, tokenizer, max_length, seed_text):
    in_text = seed_text
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
    yhat = model.predict(encoded, verbose=0)
    return yhat

In [158]:
np.sort(get_all_probs(model2, tokenizer, max_length-1, 'взгляд православный')[0])[-5:]

array([ 0.01513699,  0.01998728,  0.04498252,  0.0994028 ,  0.36261377], dtype=float32)

In [161]:
np.where(get_all_probs(model2, tokenizer, max_length-1, 'взгляд православный')[0]==0.36261377)[0][0]

1772

In [208]:
#генерируем строки на замену

inv_dict = {v: k for k, v in tokenizer.word_index.items()}
from pymystem3 import Mystem
m = Mystem()

small=1e-10
#give it lemmatized strings maybe
def get_trigram(trigram_string):
    suggestions=[]
    #for string in trigram_strings:
    prob=0.0
    bigrams=' '.join(m.lemmatize(i)[0] for i in trigram_string[:-1])
    last_word=m.lemmatize(trigram_string[-1])[0]
    prob_matrix=get_all_probs(model2, tokenizer, max_length-1, bigrams)[0]
    most_probable=np.sort(prob_matrix)[-100:]
    if last_word in tokenizer.word_index.keys():
        prob=prob_matrix[tokenizer.word_index[last_word]]
    else:
        prob=small
    if prob in most_probable:
        #pass if trigram's probability is high
        pass
    else:
        for i in most_probable[-5:]:
            ind=np.where(prob_matrix==i)[0][0]
            suggestions.append(bigrams+' '+inv_dict[ind])
    if suggestions:
        return 'возможно неверная коллокация: {}, варианты замен: {}'.format(trigram_string, suggestions)

In [210]:
get_trigram(['взгляд', 'православный', 'верующий'])

In [153]:
tokenizer.word_index['верующий']

1772

In [178]:
#тестируем
test=open('бэд.txt', encoding='utf-8').read()

In [196]:
import nltk
from nltk.util import ngrams

In [187]:
clean_text=[]
for i in test.split():
    i=i.lower()
    if p.search(i) and not p1.search(i):
        clean_text.append(i)

In [202]:
trigrams=list(ngrams(clean_text,3))

In [209]:
get_trigram(list(trigrams[10]))

"возможно неверная коллокация: ['всероссийский', 'умер', 'уже'], варианты замен: ['всероссийский умирать быть', 'всероссийский умирать г', 'всероссийский умирать до', 'всероссийский умирать якобсон', 'всероссийский умирать в']"